In [1]:
from mandala._next.imports import *

In [2]:
storage = Storage()

@op
def f(x: int) -> int:
    return x + 1

@op
def g(x: int) -> int:
    return x + 2

@op
def inc_list(things: MList[int]) -> MList[int]:
    return [x + 1 for x in things]

@op
def inc_dict(things: MDict[str, int]) -> MDict[str, int]:
    return {k: v + 1 for k, v in things.items()}

In [10]:
with storage:
    x = f(1)
    y = g(x)
    z = inc_list([x, y])
    w = inc_dict({'x': x, 'y': y})

In [11]:
storage.unwrap(w)

{'x': 3, 'y': 5}